In [81]:
import pandas as pd
import numpy as np

In [82]:
# Dataframes from Census-Data
census_column_keys = pd.read_csv("../shared_data_read_only/Census-Data/ACSST5Y2022.S2201-Column-Metadata.csv")
census_data = pd.read_csv("../shared_data_read_only/Census-Data/ACSST5Y2022.S2201-Data.csv", low_memory=False)
nonprofit_data = pd.read_csv("../shared_data_read_only/Census-Data/Nonprofit_Data_PA.csv")

In [83]:
# Dataframes from Education-Data
enrollment_demographics = pd.read_csv("../shared_data_read_only/Education-Data/2023-2024 Enrollment & Demographics.csv")
master_school_list = pd.read_csv("../shared_data_read_only/Education-Data/2023-2024 Master School List (20231003).csv")
college_matriculation = pd.read_csv("../shared_data_read_only/Education-Data/SDP_College_Matriculation_2021-2022.csv")
graduation_rates = pd.read_csv("../shared_data_read_only/Education-Data/SDP_Graduation_Rates_School_S_2023-02-15.csv")
school_metrics = pd.read_csv("../shared_data_read_only/Education-Data/SPREE_SY2122_School_Metric_Scores.csv")
student_ada = pd.read_csv("../shared_data_read_only/Education-Data/Student_ADA_Yearly.csv")

In [84]:
# Dataframes from Geography-Mapping
geography_map = pd.read_csv("../shared_data_read_only/Geography-Mapping/ZIP_TRACT_122023.csv")

In [85]:
PA_census_data = census_data[census_data['NAME'].str.contains('Pennsylvania')]
PA_census_data.loc[:, 'GEO_ID'] = PA_census_data['GEO_ID'].str.replace('1400000US', '')
PA_census_data = PA_census_data.rename({'GEO_ID' : 'TRACT'}, axis=1)
PA_census_data['TRACT'] = PA_census_data['TRACT'].astype(str)
PA_geography_map = geography_map[geography_map['USPS_ZIP_PREF_STATE'].str.contains('PA')]
PA_geography_map.loc[:, 'TRACT'] = PA_geography_map['TRACT'].astype(str)
merged_census_data = pd.merge(PA_census_data, PA_geography_map, on='TRACT', how='outer')
print(merged_census_data)
columns = ['ZIP'] + [col for col in merged_census_data.columns if col != 'ZIP']
merged_census_data['ZIP'] = merged_census_data['ZIP'].fillna(0).astype(int)
merged_census_data.head(5)

            TRACT                                             NAME  \
0     42001030101  Census Tract 301.01; Adams County; Pennsylvania   
1     42001030101  Census Tract 301.01; Adams County; Pennsylvania   
2     42001030101  Census Tract 301.01; Adams County; Pennsylvania   
3     42001030101  Census Tract 301.01; Adams County; Pennsylvania   
4     42001030103  Census Tract 301.03; Adams County; Pennsylvania   
...           ...                                              ...   
8903  42133024001   Census Tract 240.01; York County; Pennsylvania   
8904  42133024002   Census Tract 240.02; York County; Pennsylvania   
8905  42133024002   Census Tract 240.02; York County; Pennsylvania   
8906  42133024002   Census Tract 240.02; York County; Pennsylvania   
8907  42133024002   Census Tract 240.02; York County; Pennsylvania   

     S2201_C01_001E S2201_C01_001M S2201_C01_002E S2201_C01_002M  \
0              1061             90            451             81   
1              1061    

,TRACT,NAME,S2201_C01_001E,S2201_C01_001M,S2201_C01_002E,S2201_C01_002M,S2201_C01_003E,S2201_C01_003M,S2201_C01_004E,S2201_C01_004M,...,S2201_C06_038E,S2201_C06_038M,Unnamed: 458,ZIP,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,42001030101,Census Tract 301.01; Adams County; Pennsylvania,1061,90,451,81,610,99,676,75,...,55.8,8.6,NaN,17019,DILLSBURG,PA,0.006840,0.005141,0.0,0.006604
1,42001030101,Census Tract 301.01; Adams County; Pennsylvania,1061,90,451,81,610,99,676,75,...,55.8,8.6,NaN,17316,EAST BERLIN,PA,0.099727,0.054795,0.0,0.098321
2,42001030101,Census Tract 301.01; Adams County; Pennsylvania,1061,90,451,81,610,99,676,75,...,55.8,8.6,NaN,17324,GARDNERS,PA,0.016313,0.026316,0.0,0.016454
3,42001030101,Census Tract 301.01; Adams County; Pennsylvania,1061,90,451,81,610,99,676,75,...,55.8,8.6,NaN,17372,YORK SPRINGS,PA,0.406977,0.420290,0.3,0.406893
4,42001030103,Census Tract 301.03; Adams County; Pennsylvania,847,160,399,151,448,134,662,140,...,77.1,11.4,NaN,17316,EAST BERLIN,PA,0.281421,0.041096,0.1,0.275779


In [86]:
columns_to_keep = ["ZIP", "TRACT", "NAME", "S2201_C01_034E", "S2201_C03_001E"]
merged_census_data = merged_census_data[columns_to_keep]
merged_census_data.rename(columns={'S2201_C01_034E': 'MEDIAN INCOME', "S2201_C03_001E":"Food stamps"}, inplace=True)
merged_census_data.head(5)
# print(len(merged_census_data))

,ZIP,TRACT,NAME,MEDIAN INCOME,Food stamps
0,17019,42001030101,Census Tract 301.01; Adams County; Pennsylvania,80231,46
1,17316,42001030101,Census Tract 301.01; Adams County; Pennsylvania,80231,46
2,17324,42001030101,Census Tract 301.01; Adams County; Pennsylvania,80231,46
3,17372,42001030101,Census Tract 301.01; Adams County; Pennsylvania,80231,46
4,17316,42001030103,Census Tract 301.03; Adams County; Pennsylvania,111603,40


In [87]:
import json
file_path = '../Team-7/zipcodes_within_radius.json'
with open(file_path, 'r') as file:
    data = json.load(file)

zip_codes_list = [int(item['zip_code']) for item in data['zip_codes']]
merged_census_data = merged_census_data[merged_census_data['ZIP'].isin(zip_codes_list)]
print(len(merged_census_data))

1188


In [88]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# Group by ZIP and calculate the mean of all other columns
merged_census_data = merged_census_data.groupby('ZIP').mean().reset_index()

# Filter rows for ZIP codes 19140 - A and 19104 - B
zip_A = '19140'
zip_B = '19104'

# Filter rows for the ZIP codes
rowA_df = merged_census_data[merged_census_data['ZIP'] == zip_A]
rowB_df = merged_census_data[merged_census_data['ZIP'] == zip_B]

if not rowA_df.empty and not rowB_df.empty:
    # Extract values excluding ZIP
    rowA = rowA_df.iloc[0, 1:].values.reshape(1, -1)  # Make sure it's 2D
    rowB = rowB_df.iloc[0, 1:].values.reshape(1, -1)

    # Normalize the data using StandardScaler
    scaler = StandardScaler()
    scaler.fit(merged_census_data.iloc[:, 2:])  # Fit scaler to all numeric data after ZIP

    rowA_scaled = scaler.transform(rowA)  # Transform data
    rowB_scaled = scaler.transform(rowB)

    # Calculate Euclidean distance
    euclidean_dist = euclidean(rowA_scaled, rowB_scaled)

    # Calculate Cosine Similarity
    cosine_sim = cosine_similarity(rowA_scaled, rowB_scaled)[0][0]

    # Calculate Pearson Correlation
    pearson_corr, _ = pearsonr(rowA_scaled.flatten(), rowB_scaled.flatten())  # Flatten since it's a 1D correlation

    # Display the results
    print(f"Euclidean Distance between ZIP {zip_A} and {zip_B}: {euclidean_dist}")
    print(f"Cosine Similarity between ZIP {zip_A} and {zip_B}: {cosine_sim}")
    print(f"Pearson Correlation between ZIP {zip_A} and {zip_B}: {pearson_corr}")
else:
    print("One or both ZIP codes not found in the dataset. Please check the ZIP codes and try again.")

TypeError: agg function failed [how->mean,dtype->object]

In [89]:
merged_census_data['MEDIAN INCOME'] = pd.to_numeric(merged_census_data['MEDIAN INCOME'], errors='coerce')
merged_census_data['Food stamps'] = pd.to_numeric(merged_census_data['Food stamps'], errors='coerce')

# Group by 'TRACT' and calculate the mean for other columns
merged_census_data = merged_census_data.groupby('TRACT').agg({
    'ZIP':'first',
    'NAME': 'first',
    'MEDIAN INCOME': 'mean',
    'Food stamps': 'mean'
}).reset_index()
merged_census_data.head(5)

,TRACT,ZIP,NAME,MEDIAN INCOME,Food stamps
0,42017100102,19020,Census Tract 1001.02; Bucks County; Pennsylvania,77946.0,109.0
1,42017100103,19020,Census Tract 1001.03; Bucks County; Pennsylvania,64417.0,173.0
2,42017100104,19020,Census Tract 1001.04; Bucks County; Pennsylvania,53393.0,633.0
3,42017100105,19020,Census Tract 1001.05; Bucks County; Pennsylvania,92557.0,54.0
4,42017100201,19053,Census Tract 1002.01; Bucks County; Pennsylvania,90968.0,95.0


In [90]:
merged_census_data = merged_census_data.groupby('ZIP').agg({
    'MEDIAN INCOME': 'mean',
    'Food stamps': 'mean'
}).reset_index()

In [91]:
print(len(merged_census_data))

100


In [92]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# Filter rows for ZIP codes 19140 - A and 19104 - B
zip_A = '19140'
zip_B = '19104'

# Filter rows for the ZIP codes
rowA_df = merged_census_data[merged_census_data['ZIP'] == zip_A]
rowB_df = merged_census_data[merged_census_data['ZIP'] == zip_B]

if not rowA_df.empty and not rowB_df.empty:
    # Extract values
    rowA = rowA_df.iloc[0, 1:].values.astype(float)  # Start from 1 to exclude ZIP
    rowB = rowB_df.iloc[0, 1:].values.astype(float)

    # Normalize the data
    scaler = StandardScaler()
    scaler.fit(merged_census_data.iloc[:, 2:])  # Fit scaler to all numeric data
    rowA_scaled = scaler.transform([rowA])[0]  # Transform data
    rowB_scaled = scaler.transform([rowB])[0]

    # Calculate Euclidean distance
    euclidean_dist = euclidean(rowA_scaled, rowB_scaled)

    # Calculate Cosine Similarity
    cosine_sim = cosine_similarity([rowA_scaled], [rowB_scaled])[0][0]

    # Calculate Pearson Correlation
    pearson_corr, _ = pearsonr(rowA_scaled, rowB_scaled)

    # Display the results
    print(f"Euclidean Distance between ZIP {zip_A} and {zip_B}: {euclidean_dist}")
    print(f"Cosine Similarity between ZIP {zip_A} and {zip_B}: {cosine_sim}")
    print(f"Pearson Correlation between ZIP {zip_A} and {zip_B}: {pearson_corr}")
else:
    print("One or both ZIP codes not found in the dataset. Please check the ZIP codes and try again.")


One or both ZIP codes not found in the dataset. Please check the ZIP codes and try again.


In [52]:
# Dropping unnecessary rows from enrollment_demographics dataframe.
updated_enrollment_demographics = enrollment_demographics.loc[enrollment_demographics['GradeLevel'] == 'All Grades']
print(len(updated_enrollment_demographics))

331


In [53]:
updated_student_ada = student_ada.loc[student_ada['School Year'] == '2021-2022']
print(len(updated_student_ada))

214


In [54]:
# Dropping unnecessary rows from updated_graduation_rates dataframe.
print(len(graduation_rates))
graduation_rates['score'] = pd.to_numeric(graduation_rates['score'], errors='coerce')
updated_cleaned_data = graduation_rates.dropna(subset=['score'])
updated_graduation_rates = updated_cleaned_data.groupby(['schoolid_ulcs', 'schoolname', 'sector', 'rate_type', 'group', 'subgroup'])['score'].mean().reset_index()
updated_graduation_rates.rename(columns={'score': 'average_score'}, inplace=True)
updated_graduation_rates = updated_graduation_rates.loc[updated_graduation_rates['group'] == 'All Students']
updated_graduation_rates = updated_graduation_rates.loc[updated_graduation_rates['rate_type'] == '4-Year Graduation Rate']
print(len(updated_graduation_rates))
updated_graduation_rates.head(5)

26163
95


,schoolid_ulcs,schoolname,sector,rate_type,group,subgroup,average_score
0,1010,John Bartram High School,District,4-Year Graduation Rate,All Students,All Students,64.932222
36,1020,West Philadelphia High School,District,4-Year Graduation Rate,All Students,All Students,64.043333
61,1030,High School of the Future,District,4-Year Graduation Rate,All Students,All Students,80.133333
88,1050,Paul Robeson High School for Human Services,District,4-Year Graduation Rate,All Students,All Students,94.727778
118,1080,University City High School,District,4-Year Graduation Rate,All Students,All Students,12.000000


In [55]:
# converting the names of schools to lowercase
updated_enrollment_demographics['SchoolName'] = updated_enrollment_demographics['SchoolName'].str.lower()
updated_enrollment_demographics.rename(columns={'SchoolName': 'School Name'}, inplace=True)
master_school_list['School Name (ULCS)'] = master_school_list['School Name (ULCS)'].str.lower()
master_school_list.rename(columns={'School Name (ULCS)': 'School Name'}, inplace=True)
college_matriculation['School Name'] = college_matriculation['School Name'].str.lower()
updated_student_ada['School Name'] = updated_student_ada['School Name'].str.lower()
updated_graduation_rates.rename(columns={'schoolname': 'School Name'}, inplace=True)
updated_graduation_rates['School Name'] = college_matriculation['School Name'].str.lower()
updated_graduation_rates.head(5)

/tmp/ipykernel_123807/1720002524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_enrollment_demographics['SchoolName'] = updated_enrollment_demographics['SchoolName'].str.lower()
/tmp/ipykernel_123807/1720002524.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_enrollment_demographics.rename(columns={'SchoolName': 'School Name'}, inplace=True)
/tmp/ipykernel_123807/1720002524.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

,schoolid_ulcs,School Name,sector,rate_type,group,subgroup,average_score
0,1010,john bartram high school,District,4-Year Graduation Rate,All Students,All Students,64.932222
36,1020,central high school,District,4-Year Graduation Rate,All Students,All Students,64.043333
61,1030,NaN,District,4-Year Graduation Rate,All Students,All Students,80.133333
88,1050,NaN,District,4-Year Graduation Rate,All Students,All Students,94.727778
118,1080,NaN,District,4-Year Graduation Rate,All Students,All Students,12.000000


In [56]:
# Merging the four dataframes from the education-data
education_merged_df = pd.merge(updated_enrollment_demographics, master_school_list, on='School Name', how='outer')
education_merged_df = pd.merge(education_merged_df, college_matriculation, on='School Name', how='outer')
education_merged_df = pd.merge(education_merged_df, updated_student_ada, on='School Name', how='outer')
education_merged_df = pd.merge(education_merged_df, updated_graduation_rates, on='School Name', how='outer')
education_merged_df.head(10)

,SchoolYear,Sector_x,SubSector,ulcscode,School Name,LearningNetwork,GradeLevel,StudentEnrollment,ELCount,ELPCT,...,School Year_y,Sector_y,ULCS Code,Average Daily Attendance (YTD),schoolid_ulcs,sector,rate_type,group,subgroup,average_score
0,2023-2024,District,District,1010.0,john bartram high school,Network 13,All Grades,526.0,114.0,21.67,...,2021-2022,District,1010.0,76.70,1010.0,District,4-Year Graduation Rate,All Students,All Students,64.932222
1,2023-2024,District,District,1020.0,west philadelphia high school,Network 13,All Grades,601.0,30.0,4.99,...,2021-2022,District,1020.0,85.87,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-2024,District,District,1030.0,high school of the future,Innovation,All Grades,560.0,11.0,1.96,...,2021-2022,District,1030.0,83.85,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-2024,District,District,1050.0,paul robeson high school for human services,Network 1,All Grades,291.0,15.0,5.15,...,2021-2022,District,1050.0,92.37,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-2024,District,District,1100.0,william l. sayre high school,Network 13,All Grades,426.0,16.0,3.76,...,2021-2022,District,1100.0,81.19,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-2024,District,District,1130.0,william t. tilden school,Network 10,All Grades,281.0,59.0,21.00,...,2021-2022,District,1130.0,88.33,NaN,NaN,NaN,NaN,NaN,NaN
6,2023-2024,District,District,1190.0,motivation high school,Network 1,All Grades,239.0,24.0,10.04,...,2021-2022,District,1190.0,92.34,NaN,NaN,NaN,NaN,NaN,NaN
7,2023-2024,District,District,1200.0,john barry school,Network 5,All Grades,493.0,9.0,1.83,...,2021-2022,District,1200.0,80.45,NaN,NaN,NaN,NaN,NaN,NaN
8,2023-2024,District,District,1230.0,william c. bryant school,Acceleration,All Grades,341.0,18.0,5.28,...,2021-2022,District,1230.0,80.42,NaN,NaN,NaN,NaN,NaN,NaN
9,2023-2024,District,District,1250.0,joseph w. catharine school,Network 12,All Grades,398.0,98.0,24.62,...,2021-2022,District,1250.0,87.39,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
columns_to_keep = ["StudentEnrollment", "Zip Code", "Average Daily Attendance (YTD)", "average_score"]
education_merged_df = education_merged_df[columns_to_keep]
education_merged_df = education_merged_df.dropna(subset=['Zip Code'])
education_merged_df.head(10)

,StudentEnrollment,Zip Code,Average Daily Attendance (YTD),average_score
0,526.0,19142.0,76.70,64.932222
1,601.0,19139.0,85.87,NaN
2,560.0,19104.0,83.85,NaN
6,239.0,19143.0,92.34,NaN
8,341.0,19143.0,80.42,NaN
9,398.0,19142.0,87.39,NaN
10,267.0,19143.0,82.53,NaN
12,354.0,19139.0,81.56,NaN
14,461.0,19139.0,90.51,NaN
15,248.0,19143.0,84.69,NaN


In [58]:
print(len(education_merged_df))

329


In [59]:
education_merged_df = education_merged_df.groupby('Zip Code').mean().reset_index()

In [60]:
print(len(education_merged_df))

44


In [61]:
education_merged_df.head(5)

,Zip Code,StudentEnrollment,Average Daily Attendance (YTD),average_score
0,19103.0,786.333333,96.170,NaN
1,19104.0,246.200000,87.455,NaN
2,19106.0,401.000000,90.800,NaN
3,19111.0,1049.625000,89.332,NaN
4,19114.0,NaN,NaN,NaN


In [62]:
education_merged_df.rename(columns={'average_score': 'graduation_rate', 'Zip Code': "ZIP"}, inplace=True)
education_merged_df.head(5)

,ZIP,StudentEnrollment,Average Daily Attendance (YTD),graduation_rate
0,19103.0,786.333333,96.170,NaN
1,19104.0,246.200000,87.455,NaN
2,19106.0,401.000000,90.800,NaN
3,19111.0,1049.625000,89.332,NaN
4,19114.0,NaN,NaN,NaN


In [64]:
final_dataframe = pd.merge(education_merged_df, merged_census_data, on='ZIP', how='outer')
print(final_dataframe)

         ZIP  StudentEnrollment  Average Daily Attendance (YTD)  \
0    19103.0         786.333333                          96.170   
1    19104.0         246.200000                          87.455   
2    19106.0         401.000000                          90.800   
3    19111.0        1049.625000                          89.332   
4    19114.0                NaN                             NaN   
..       ...                ...                             ...   
96   19409.0                NaN                             NaN   
97   19422.0                NaN                             NaN   
98   19428.0                NaN                             NaN   
99   19444.0                NaN                             NaN   
100  19462.0                NaN                             NaN   

     graduation_rate  MEDIAN INCOME  Food stamps  
0                NaN   97153.500000   111.125000  
1                NaN   40522.650000   328.913043  
2                NaN  122808.777778    76.

In [65]:
final_dataframe.head(5)

,ZIP,StudentEnrollment,Average Daily Attendance (YTD),graduation_rate,MEDIAN INCOME,Food stamps
0,19103.0,786.333333,96.170,NaN,97153.500000,111.125000
1,19104.0,246.200000,87.455,NaN,40522.650000,328.913043
2,19106.0,401.000000,90.800,NaN,122808.777778,76.777778
3,19111.0,1049.625000,89.332,NaN,58823.823529,481.222222
4,19114.0,NaN,NaN,NaN,75014.272727,258.250000


In [66]:
final_dataframe = final_dataframe.drop(['graduation_rate'], axis=1)
final_dataframe.head(5)

,ZIP,StudentEnrollment,Average Daily Attendance (YTD),MEDIAN INCOME,Food stamps
0,19103.0,786.333333,96.170,97153.500000,111.125000
1,19104.0,246.200000,87.455,40522.650000,328.913043
2,19106.0,401.000000,90.800,122808.777778,76.777778
3,19111.0,1049.625000,89.332,58823.823529,481.222222
4,19114.0,NaN,NaN,75014.272727,258.250000


In [69]:
json_str = final_dataframe.to_json(orient='records')

# Parse the JSON string into a Python object
parsed = json.loads(json_str)

# Dump the Python object into a JSON file, formatted with indentation
with open('final_dataframe.json', 'w') as f:
    json.dump(parsed, f, indent=4)

In [93]:
import os 

os.getcwd()

'/home/jupyter-user-2/Team-7'